In [ ]:
# set up and test simple commands with LLM 
# swaped OpenAI for Mistral due to expired free tier quota

In [2]:
# import mistral 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [3]:
import os

In [4]:
# get API key stored in env variable
api_key = os.getenv("MISTRAL_KEY")

In [5]:
client = MistralClient(api_key=api_key)

In [6]:
# ask it a question
response = client.chat(
    model = 'mistral-tiny',
    messages = [ChatMessage(role="user", content="Is it too late to join the course?")],
)

In [7]:
print(response.choices[0].message.content)

I'm sorry for the confusion, but I'll need a bit more information to provide an accurate answer. The question of whether it's too late to join a course depends on the specific course you're interested in. Some courses may have strict enrollment deadlines, while others might allow students to join even after the start date.

If you have the details of the course, such as its name, institution, or a link to its website, I'd be happy to help you check their enrollment policies. Alternatively, you could contact the course provider directly to ask about their enrollment deadlines and availability.


In [8]:
# set up a search using minisearch from Alexei Grigorev
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 18:58:06--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 18:58:06 (12.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [9]:
import minsearch

In [10]:
import json

In [13]:
# get the document
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-30 19:00:47--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-30 19:00:47 (40.3 MB/s) - ‘documents.json’ saved [658332/658332]



In [14]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [15]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [17]:
# index (fit the index to) the documents
index.fit(documents)

In [18]:
# some querry (question) 
query = 'the course has already started, can I still enroll?'

In [21]:
# let's search but make some fields more important and some less so
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query, 
    filter_dict = {'course': 'data-engineering-zoomcamp'}, 
    boost_dict = boost, 
    # filter_dict = {'course': 'data-engineering-zoomcamp'}, # if we want to search only for specific course 
    num_results = 5
)

In [22]:
print(results)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}, {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. Th

In [ ]:
# TODO: generating with OpenAI (Mistral), RAG flow, Elasticsearch, HW